In [ ]:
import os
import sys
import logging
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, Any, List, Generator

In [ ]:
# package imports
from vsim import electorate, candidates, simulation, voting_system

# Analysing proportional voting systems

We'll be using the case study of the Swedish parlimentary election systems, namely that there exists n-parties, voters vote for the preferred option (using the distance method we described earlier). The difference is the "winner" is not as well defined. 

In [8]:
# setup helper for generating experiments (basically all sets of params to run through sim)
def generate_experiment_matrix(experiments: Dict[str, List[Any]]):
    return (dict(zip(experiments, x)) for x in itertools.product(*experiments.values()))

experiments = {
    "electorate_size": [10_000, 25_000, 50_000],
    "clusters": [1, 2, 5, 10],
    "cluster_std": [1, 2, 3],
    "electoral_system": ["proportional"],
    "apathy_prob": [0.0, 0.01, 0.05],
    "candidates": [2, 4, 5, 9], # akin to parties in the proportional case
    "issues": [2, 5, 10, 25],
    "seats": [150, 179, 349, 736], # dutch, danish, swedish, german parliment(s) seats
    "min_share_threshold": [0, 0.04, 0.10] # dutch, swedish, and turkish thresholds
}

records = []
columns = [*experiments.keys(), "unweighted_fairness", "weighted_fairness"]
experiment_matrix = list(generate_experiment_matrix(experiments))[:100]

for params in tqdm(experiment_matrix):

    # setup objects
    voters = electorate.setup_electorate(**params)
    parties = candidates.setup_candidates(**params, electorate=voters)
    system = voting_system.setup_voting_system(name=params["electoral_system"], **params)

    # simulate elections
    sim = simulation.VotingSimulator(electorate=voters, candidates=parties, system=system, log=logging.getLogger())

    result = sim.run()
    result_record = { 
        "fairness": round(result.measured_fairness, 3),
        "weighted_fairness": round(result.measured_weighted_fairness, 3),
        **params 
    }
    result_record = {k: v for k, v in result_record.items() if k in columns}
    records.append(result_record)

100%|████████████████████████████████████████████| 100/100 [00:08<00:00, 11.76it/s]


In [9]:
results_df = pd.DataFrame.from_records(records)
results_df

,fairness,weighted_fairness,electorate_size,clusters,cluster_std,electoral_system,apathy_prob,candidates,issues,seats,min_share_threshold
0,5.014,5.014,10000,1,1,proportional,0.0,2,2,150,0.00
1,4.826,4.826,10000,1,1,proportional,0.0,2,2,150,0.04
2,3.596,3.575,10000,1,1,proportional,0.0,2,2,150,0.10
3,5.464,5.395,10000,1,1,proportional,0.0,2,2,179,0.00
4,2.137,2.137,10000,1,1,proportional,0.0,2,2,179,0.04
...,...,...,...,...,...,...,...,...,...,...,...
95,5.365,5.358,10000,1,1,proportional,0.0,4,25,736,0.10
96,5.014,4.376,10000,1,1,proportional,0.0,5,2,150,0.00
97,7.078,5.745,10000,1,1,proportional,0.0,5,2,150,0.04
98,7.387,5.289,10000,1,1,proportional,0.0,5,2,150,0.10


##### Store data or read from already performed experiment files

In [ ]:
filepath = "data/proportional-vs-proportional-df5ec973-c180-40ef-a410-1490716cd4e2.csv"